In [1]:
import pandas as pd
import hashlib

# Przykładowy DataFrame z danymi wrażliwymi
df = pd.DataFrame({
    'imie': ['Anna', 'Piotr', 'Jan'],
    'nazwisko': ['Kowalska', 'Nowak', 'Wiśniewski'],
    'pesel': ['44051401359', '80010112345', '91050609876'],
    'miasto': ['Warszawa', 'Kraków', 'Gdańsk'],
    'wiek': [34, 45, 29]
})

# Funkcja do maskowania (pseudonimizacji) danych – np. przez hash SHA-256
def pseudonimizuj_wartosc(wartosc):
    return hashlib.sha256(wartosc.encode()).hexdigest()

# Maskowanie kolumn imie, nazwisko i pesel
for kolumna in ['imie', 'nazwisko', 'pesel']:
    df[kolumna] = df[kolumna].apply(pseudonimizuj_wartosc)

print("Zanonimizowane dane:\n")
print(df)

Zanonimizowane dane:

                                                imie  \
0  bf4fccd616251b678c56b9cb7a46819b1266853c180637...   
1  923818e5629f40f9417e101c111f341dfe2956e5388284...   
2  5c5db120cb11bee138ff3143edcbedaead684de7a0ba14...   

                                            nazwisko  \
0  3d12462270c5311e44490eb0c98ded121382fff098aa93...   
1  2add3f54e40018f50af15b7af7a124ee0f006adae3159f...   
2  b4ef58a777a294cd151e5642d5c0756d049dbac1855232...   

                                               pesel    miasto  wiek  
0  ca7e084373515292389b86d9d1b1f155852b0f35b62246...  Warszawa    34  
1  e64412ceb8f5ff659889cdbcae3b6a23c56a1688b7fa3d...    Kraków    45  
2  f1369d2b366543f5480a42df1fa4e102617e4c91de93c6...    Gdańsk    29  


In [2]:
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 137.6 MB/s eta 0:00:00


In [3]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
import base64

# Dane do zaszyfrowania
dane = "To są dane medyczne pacjenta: Jan Kowalski, 45 lat."

# Wygeneruj losowy 16-bajtowy klucz AES (dla AES-128)
klucz = get_random_bytes(16)
print("Klucz (hex):", klucz.hex())

# Inicjalizuj szyfrowanie w trybie CBC
cipher = AES.new(klucz, AES.MODE_CBC)
iv = cipher.iv  # wektor inicjalizujący

# Zakoduj i zaszyfruj dane
dane_zaszyfrowane = cipher.encrypt(pad(dane.encode(), AES.block_size))
dane_zaszyfrowane_b64 = base64.b64encode(iv + dane_zaszyfrowane)

print("\nZaszyfrowane dane (base64):", dane_zaszyfrowane_b64.decode())

# Deszyfrowanie
dane_zaszyfrowane_full = base64.b64decode(dane_zaszyfrowane_b64)
iv_odczytane = dane_zaszyfrowane_full[:AES.block_size]
cipher_dec = AES.new(klucz, AES.MODE_CBC, iv=iv_odczytane)
dane_odszyfrowane = unpad(cipher_dec.decrypt(dane_zaszyfrowane_full[AES.block_size:]), AES.block_size)

print("\nOdszyfrowane dane:", dane_odszyfrowane.decode())

Klucz (hex): d61e348ee92fae0377256065304b64ac

Zaszyfrowane dane (base64): mF5xqG8jIr42z+OjTR9sB9AnoLbGoCeOseDpcT7DPDmFd49cO4jDycaCzpjsz9iSCJTZOp5qUOx+5a31avJTocMK8votJuqQF0DqyKCEUYo=

Odszyfrowane dane: To są dane medyczne pacjenta: Jan Kowalski, 45 lat.
